# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

Как это сделать для всех пользователей сразу:

In [ ]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()


In [ ]:
# можно итоги добавить
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

### Упражнение
Какой пользователь выставил больше всех пятерок?

### Фильтры и вычисления с помощью loc и iloc

In [3]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [ ]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

In [ ]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, -3]].head()

In [ ]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

In [ ]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

In [ ]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

### Скорость метода loc
Посчитаем средний рейтинг разных жанров. Метод merge взят из материалов следующего занятия ¯ \ _ (ツ) _ / ¯

In [5]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [ ]:
joined['Adventure'] = joined.apply(lambda row: row.rating if 'Adventure' in row.genres else None, axis=1)
joined.head()

Можно и через loc

In [ ]:
joined['Adventure'] = joined.loc[joined.genres.str.contains('Adventure'), 'rating']
joined.head()

Для замера времени сделаем эти операции для набор жанров

In [ ]:
genres = ['Adventure', 'Animation', 'Children', 'Drama', 'Musical', 'Thriller']

In [ ]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.apply(lambda row: row.rating if genre in row.genres else None, axis=1)

In [4]:
%%time
joined = ratings.merge(movies, how='left', on='movieId')

for genre in genres:
    joined[genre] = joined.loc[joined.genres.str.contains(genre), 'rating']

NameError: name 'movies' is not defined

### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [28]:

log.loc[(log.traffic_source == 'google') | (log.traffic_source == 'yandex'), 'traffic_type'] = 'organic'
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

In [ ]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

### Дата и время в pandas

In [ ]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

In [ ]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

In [ ]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [ ]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

### Методы работы со строками

In [ ]:
stats = pd.read_csv('keywords.csv')
stats.head()

Проверка наличия подстроки в строке в питоне:

In [ ]:
'охотник' in 'каждый охотник желает знать...'

Аналог в pandas:

In [ ]:
stats[stats.keyword.str.contains('охотник')].head()

[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [ ]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

### replace

In [ ]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

Аналог в pandas на запросах про сериалы:

In [29]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

NameError: name 'stats' is not defined

In [ ]:
serial.keyword.str.replace('сериалы', 'книги').head()

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [ ]:
from pymystem3 import Mystem

In [ ]:
search = 'курс гривны к рублю рубли рублях'

In [ ]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

In [ ]:
' '.join(lemmas)

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [ ]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})